In [1]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
import biogeme.messaging as msg
from biogeme.expressions import Beta, DefineVariable, bioDraws, log, MonteCarlo, PanelLikelihoodTrajectory

df = pd.read_csv(r'D:\programming\JupyterNotebook Files\ISTTT second round\panel_cluster\commuter_data1231_proportion.csv').sort_values(by='GRANT_CARD_CODE')

#df = df[~(df['TRIP_ORIGIN_LOCATION'].isin([151019789])&df['Label'].isin([1,2]))]
df = df[~(df['GRANT_CARD_CODE'].isin([ 12745537, 14076463, 14927887, 17561919,
         20069895, 20124584, 23852621, 24135919,
          28427292, 39748238, 49251286, 51450840, 53780691,
       75451166, 80224716, 80359935, 80563115, 80788740, 81960595,
        21388079, 29546391, 29577400,   44847431,
       49011558, 71414486,  83623399, 85893572]))]

#df = df[~(df['TRIP_ORIGIN_LOCATION'].isin([151019787])&df['Label'].isin([2,5]))]
df = df[~(df['GRANT_CARD_CODE'].isin([  16458937, 16741497,  
       22497858, 22954095, 25830639,
         39268819, 
       43084096,    45441692, 45985977, 50478454,
         71196235, 71415417, 75235891, 
       80405151, 80642024, 80643766,  
        83614140]))]

#df = df[~(df['TRIP_ORIGIN_LOCATION'].isin([151019785])&df['Label'].isin([0,3,4]))]
df = df[~(df['GRANT_CARD_CODE'].isin([ 12331804, 13072055, 13363894, 
       18209619,  20798378, 21469212,  
        23366265, 24045595, 24929727, 25339404, 
       26910196,    29372711, 29557754, 
       33204818,    46891443, 48144104,
            84533927,
        86161427    ]))]

#df = df[~(df['TRIP_ORIGIN_LOCATION'].isin([151019784])&df['Label'].isin([6]))]
df = df[~(df['GRANT_CARD_CODE'].isin([ 16636519, 19633179,  43062163, 68160074]))]


df = df[~(df['GRANT_CARD_CODE'].isin([17001355, 80686620]))]
df = df[~(df['GRANT_CARD_CODE'].isin([14622752, 25890510]))]
df = df[~(df['GRANT_CARD_CODE'].isin([12748879, 86638694]))]
df = df[~(df['GRANT_CARD_CODE'].isin([14596920,14878274,43025843,60206904,60609222]))]

df

,Label_old,ARRIVETIME_PREFER_old,Label,CHOICE,ARRIVETIME_PREFER,GRANT_CARD_CODE,TRIP_ORIGIN_LOCATION,进站时间,到达时间,出行时长,...,PCC_37,PCC_38,PCC_39,PCC_40,PCC_41,PCC_42,PCC_43,PCC_44,PCC_45,PCC_46
0,0,30300,9,19,30300,1229463,151019784,28680,30249,1569,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,0,30300,9,16,30300,1229463,151019784,28200,29538,1338,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,0,30300,9,22,30300,1229463,151019784,29700,30960,1260,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,0,30300,9,16,30300,1229463,151019784,28200,29534,1334,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0,30300,9,16,30300,1229463,151019784,28140,29536,1396,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7421,1,30900,3,24,30900,89911490,151019787,29340,31297,1957,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7420,1,30900,3,24,30900,89911490,151019787,29460,31305,1845,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7432,1,30900,3,21,30900,89911490,151019787,29040,30760,1720,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7425,1,30900,3,23,30900,89911490,151019787,29460,31282,1822,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [2]:
database = db.Database('panel_commuter_data', df)
globals().update(database.variables)
B_alpha = Beta("B_alpha",0,None,None,0)
B_beta = Beta("B_beta",0,None,None,0)
B_gamma = Beta("B_gamma",0,None,None,0)
B_g0 = Beta("B_g0",0,None,None,0)

B_alpha_S = Beta('B_alpha_S',0, None, None, 0)
B_alpha_RND = B_alpha + B_alpha_S * bioDraws('B_alpha_RND','NORMAL')
B_beta_S = Beta('B_beta_S',0, None, None, 0)
B_beta_RND = B_beta + B_beta_S * bioDraws('B_beta_RND','NORMAL')
B_gamma_S = Beta('B_gamma_S',0, None, None, 0)
B_gamma_RND = B_gamma + B_gamma_S * bioDraws('B_gamma_RND','NORMAL')
B_g0_S = Beta('B_g0_S',0, None, None, 0)
B_g0_RND = B_g0 + B_g0_S * bioDraws('B_g0_RND','NORMAL')

V1 = B_alpha_RND * TRIPTIME_TYPE_1 + B_beta_RND * TE_1 + B_gamma_RND * TL_1 + B_g0_RND * crowding_cost_1 #+ B_pcc_RND * PCC_1
V2 = B_alpha_RND * TRIPTIME_TYPE_2 + B_beta_RND * TE_2 + B_gamma_RND * TL_2 + B_g0_RND * crowding_cost_2 #+ B_pcc_RND * PCC_2 
V3 = B_alpha_RND * TRIPTIME_TYPE_3 + B_beta_RND * TE_3 + B_gamma_RND * TL_3 + B_g0_RND * crowding_cost_3 #+ B_pcc_RND * PCC_3
V4 = B_alpha_RND * TRIPTIME_TYPE_4 + B_beta_RND * TE_4 + B_gamma_RND * TL_4 + B_g0_RND * crowding_cost_4 #+ B_pcc_RND * PCC_4
V5 = B_alpha_RND * TRIPTIME_TYPE_5 + B_beta_RND * TE_5 + B_gamma_RND * TL_5 + B_g0_RND * crowding_cost_5 #+ B_pcc_RND * PCC_5
V6 = B_alpha_RND * TRIPTIME_TYPE_6 + B_beta_RND * TE_6 + B_gamma_RND * TL_6 + B_g0_RND * crowding_cost_6 #+ B_pcc_RND * PCC_6 
V7 = B_alpha_RND * TRIPTIME_TYPE_7 + B_beta_RND * TE_7 + B_gamma_RND * TL_7 + B_g0_RND * crowding_cost_7 #+ B_pcc_RND * PCC_7
V8 = B_alpha_RND * TRIPTIME_TYPE_8 + B_beta_RND * TE_8 + B_gamma_RND * TL_8 + B_g0_RND * crowding_cost_8 #+ B_pcc_RND * PCC_8
V9 = B_alpha_RND * TRIPTIME_TYPE_9 + B_beta_RND * TE_9 + B_gamma_RND * TL_9 + B_g0_RND * crowding_cost_9 #+ B_pcc_RND * PCC_9 
V10 = B_alpha_RND * TRIPTIME_TYPE_10 + B_beta_RND * TE_10 + B_gamma_RND * TL_10 + B_g0_RND * crowding_cost_10 #+ B_pcc_RND * PCC_10 
V11 = B_alpha_RND * TRIPTIME_TYPE_11 + B_beta_RND * TE_11 + B_gamma_RND * TL_11 + B_g0_RND * crowding_cost_11 #+ B_pcc_RND * PCC_11 
V12 = B_alpha_RND * TRIPTIME_TYPE_12 + B_beta_RND * TE_12 + B_gamma_RND * TL_12 + B_g0_RND * crowding_cost_12 #+ B_pcc_RND * PCC_12 
V13 = B_alpha_RND * TRIPTIME_TYPE_13 + B_beta_RND * TE_13 + B_gamma_RND * TL_13 + B_g0_RND * crowding_cost_13 #+ B_pcc_RND * PCC_13 
V14 = B_alpha_RND * TRIPTIME_TYPE_14 + B_beta_RND * TE_14 + B_gamma_RND * TL_14 + B_g0_RND * crowding_cost_14 #+ B_pcc_RND * PCC_14 
V15 = B_alpha_RND * TRIPTIME_TYPE_15 + B_beta_RND * TE_15 + B_gamma_RND * TL_15 + B_g0_RND * crowding_cost_15 #+ B_pcc_RND * PCC_15 
V16 = B_alpha_RND * TRIPTIME_TYPE_16 + B_beta_RND * TE_16 + B_gamma_RND * TL_16 + B_g0_RND * crowding_cost_16 #+ B_pcc_RND * PCC_16 
V17 = B_alpha_RND * TRIPTIME_TYPE_17 + B_beta_RND * TE_17 + B_gamma_RND * TL_17 + B_g0_RND * crowding_cost_17 #+ B_pcc_RND * PCC_17 
V18 = B_alpha_RND * TRIPTIME_TYPE_18 + B_beta_RND * TE_18 + B_gamma_RND * TL_18 + B_g0_RND * crowding_cost_18 #+ B_pcc_RND * PCC_18 
V19 = B_alpha_RND * TRIPTIME_TYPE_19 + B_beta_RND * TE_19 + B_gamma_RND * TL_19 + B_g0_RND * crowding_cost_19 #+ B_pcc_RND * PCC_19 
V20 = B_alpha_RND * TRIPTIME_TYPE_20 + B_beta_RND * TE_20 + B_gamma_RND * TL_20 + B_g0_RND * crowding_cost_20 #+ B_pcc_RND * PCC_20 
V21 = B_alpha_RND * TRIPTIME_TYPE_21 + B_beta_RND * TE_21 + B_gamma_RND * TL_21 + B_g0_RND * crowding_cost_21 #+ B_pcc_RND * PCC_21 
V22 = B_alpha_RND * TRIPTIME_TYPE_22 + B_beta_RND * TE_22 + B_gamma_RND * TL_22 + B_g0_RND * crowding_cost_22 #+ B_pcc_RND * PCC_22 
V23 = B_alpha_RND * TRIPTIME_TYPE_23 + B_beta_RND * TE_23 + B_gamma_RND * TL_23 + B_g0_RND * crowding_cost_23 #+ B_pcc_RND * PCC_23 
V24 = B_alpha_RND * TRIPTIME_TYPE_24 + B_beta_RND * TE_24 + B_gamma_RND * TL_24 + B_g0_RND * crowding_cost_24 #+ B_pcc_RND * PCC_24 
V25 = B_alpha_RND * TRIPTIME_TYPE_25 + B_beta_RND * TE_25 + B_gamma_RND * TL_25 + B_g0_RND * crowding_cost_25 #+ B_pcc_RND * PCC_25 
V26 = B_alpha_RND * TRIPTIME_TYPE_26 + B_beta_RND * TE_26 + B_gamma_RND * TL_26 + B_g0_RND * crowding_cost_26 #+ B_pcc_RND * PCC_26 
V27 = B_alpha_RND * TRIPTIME_TYPE_27 + B_beta_RND * TE_27 + B_gamma_RND * TL_27 + B_g0_RND * crowding_cost_27 #+ B_pcc_RND * PCC_27 
V28 = B_alpha_RND * TRIPTIME_TYPE_28 + B_beta_RND * TE_28 + B_gamma_RND * TL_28 + B_g0_RND * crowding_cost_28 #+ B_pcc_RND * PCC_28 
V29 = B_alpha_RND * TRIPTIME_TYPE_29 + B_beta_RND * TE_29 + B_gamma_RND * TL_29 + B_g0_RND * crowding_cost_29 #+ B_pcc_RND * PCC_29 
V30 = B_alpha_RND * TRIPTIME_TYPE_30 + B_beta_RND * TE_30 + B_gamma_RND * TL_30 + B_g0_RND * crowding_cost_30 #+ B_pcc_RND * PCC_30 
V31 = B_alpha_RND * TRIPTIME_TYPE_31 + B_beta_RND * TE_31 + B_gamma_RND * TL_31 + B_g0_RND * crowding_cost_31 #+ B_pcc_RND * PCC_31 
V32 = B_alpha_RND * TRIPTIME_TYPE_32 + B_beta_RND * TE_32 + B_gamma_RND * TL_32 + B_g0_RND * crowding_cost_32 #+ B_pcc_RND * PCC_32 
V33 = B_alpha_RND * TRIPTIME_TYPE_33 + B_beta_RND * TE_33 + B_gamma_RND * TL_33 + B_g0_RND * crowding_cost_33 #+ B_pcc_RND * PCC_33 
V34 = B_alpha_RND * TRIPTIME_TYPE_34 + B_beta_RND * TE_34 + B_gamma_RND * TL_34 + B_g0_RND * crowding_cost_34 #+ B_pcc_RND * PCC_34 
V35 = B_alpha_RND * TRIPTIME_TYPE_35 + B_beta_RND * TE_35 + B_gamma_RND * TL_35 + B_g0_RND * crowding_cost_35 #+ B_pcc_RND * PCC_35 
V36 = B_alpha_RND * TRIPTIME_TYPE_36 + B_beta_RND * TE_36 + B_gamma_RND * TL_36 + B_g0_RND * crowding_cost_36 #+ B_pcc_RND * PCC_36 
V37 = B_alpha_RND * TRIPTIME_TYPE_37 + B_beta_RND * TE_37 + B_gamma_RND * TL_37 + B_g0_RND * crowding_cost_37 #+ B_pcc_RND * PCC_37 
V38 = B_alpha_RND * TRIPTIME_TYPE_38 + B_beta_RND * TE_38 + B_gamma_RND * TL_38 + B_g0_RND * crowding_cost_38 #+ B_pcc_RND * PCC_38 
V39 = B_alpha_RND * TRIPTIME_TYPE_39 + B_beta_RND * TE_39 + B_gamma_RND * TL_39 + B_g0_RND * crowding_cost_39 #+ B_pcc_RND * PCC_39 
V40 = B_alpha_RND * TRIPTIME_TYPE_40 + B_beta_RND * TE_40 + B_gamma_RND * TL_40 + B_g0_RND * crowding_cost_40 #+ B_pcc_RND * PCC_40 
V41 = B_alpha_RND * TRIPTIME_TYPE_41 + B_beta_RND * TE_41 + B_gamma_RND * TL_41 + B_g0_RND * crowding_cost_41 #+ B_pcc_RND * PCC_41 
V42 = B_alpha_RND * TRIPTIME_TYPE_42 + B_beta_RND * TE_42 + B_gamma_RND * TL_42 + B_g0_RND * crowding_cost_42 #+ B_pcc_RND * PCC_42 
V43 = B_alpha_RND * TRIPTIME_TYPE_43 + B_beta_RND * TE_43 + B_gamma_RND * TL_43 + B_g0_RND * crowding_cost_43 #+ B_pcc_RND * PCC_43 
V44 = B_alpha_RND * TRIPTIME_TYPE_44 + B_beta_RND * TE_44 + B_gamma_RND * TL_44 + B_g0_RND * crowding_cost_44 #+ B_pcc_RND * PCC_44 
V45 = B_alpha_RND * TRIPTIME_TYPE_45 + B_beta_RND * TE_45 + B_gamma_RND * TL_45 + B_g0_RND * crowding_cost_45 #+ B_pcc_RND * PCC_45 
V46 = B_alpha_RND * TRIPTIME_TYPE_46 + B_beta_RND * TE_46 + B_gamma_RND * TL_46 + B_g0_RND * crowding_cost_46 #+ B_pcc_RND * PCC_46 

V={1: -V1, 2: -V2, 3: -V3, 4: -V4, 5: -V5, 6: -V6, 7: -V7, 8: -V8, 9: -V9, 10: -V10, 11: -V11, 12: -V12, 13: -V13, 14: -V14, 15: -V15, 16: -V16, 17: -V17, 18: -V18, 19: -V19, 20: -V20, 21: -V21, 22: -V22, 23: -V23, 24: -V24, 25: -V25, 26: -V26, 27: -V27, 28: -V28, 29: -V29, 30: -V30, 31: -V31, 32:-V32, 33: -V33, 34: -V34, 35: -V35, 36:-V36, 37: -V37, 38: -V38, 39: -V39, 40: -V40, 41: -V41, 42: -V42, 43: -V43, 44: -V44, 45: -V45, 46: -V46}
av={1:AV_1,2:AV_2,3:AV_3,4:AV_4,5:AV_5,6:AV_6,7:AV_7,8:AV_8,9:AV_9,10:AV_10,11:AV_11,12:AV_12,13:AV_13,14:AV_14,15:AV_15,16:AV_16,17:AV_17,18:AV_18,19:AV_19,20:AV_20,21:AV_21,22:AV_22,23:AV_23,24:AV_24,25:AV_25,26:AV_26,27:AV_27,28:AV_28,29:AV_29,30:AV_30,31:AV_31,32:AV_32,33:AV_33,34:AV_34,35:AV_35,36:AV_36,37:AV_37,38:AV_38,39:AV_39,40:AV_40,41:AV_41,42:AV_42,43:AV_43,44:AV_44,45:AV_45,46:AV_46}

prob = models.logit(V, av, CHOICE)
logprob = log(MonteCarlo(prob))
logger = msg.bioMessage()
logger.setGeneral()
userNotes = (
    'Example of a mixture of logit models with three alternatives, '
    'approximated using Monte-Carlo integration integration with Halton draws.'
)
# Create the Biogeme object
biogeme = bio.BIOGEME(database, logprob, numberOfDraws = 1000, userNotes=userNotes)
biogeme.modelName = 'Mixture0103'

[23:03:15] < General >   Remove 105 unused variables from the database as only 231 are used.


In [4]:
results = biogeme.estimate()
pandasResults = results.getEstimatedParameters()
print(pandasResults)

[23:03:55] < General >   *** Initial values of the parameters are obtained from the file __Mixture0103.iter
[23:03:55] < Warning >   Cannot read file __Mixture0103.iter. Statement is ignored.
[23:14:16] < General >   Log likelihood (N = 6356):  -19224.69 Gradient norm:      5e+06 Hessian norm:       2e+10 
[23:30:31] < General >   Log likelihood (N = 6356):  -15164.72 Gradient norm:      1e+06 Hessian norm:       1e+10 
[23:46:23] < General >   Log likelihood (N = 6356):  -14669.71 Gradient norm:      3e+05 Hessian norm:       9e+09 
[00:01:39] < General >   Log likelihood (N = 6356):  -14638.19 Gradient norm:      2e+04 Hessian norm:       8e+09 
[01:38:12] < General >   Log likelihood (N = 6356):  -14636.89 Gradient norm:      3e+04 Hessian norm:       8e+09 
[01:57:33] < General >   Log likelihood (N = 6356):  -14634.37 Gradient norm:      4e+04 Hessian norm:       8e+09 
[02:15:26] < General >   Log likelihood (N = 6356):  -14630.25 Gradient norm:      5e+04 Hessian norm:       8e+

In [5]:
print(results)


Results for model Mixture0103
Output file (HTML):			Mixture0103.html
Nbr of parameters:		8
Sample size:			6356
Excluded data:			0
Init log likelihood:		-19224.69
Final log likelihood:		-14622.92
Likelihood ratio test (init):		9203.543
Rho square (init):			0.239
Rho bar square (init):			0.239
Akaike Information Criterion:	29261.84
Bayesian Information Criterion:	29315.9
Final gradient norm:		0.7399687
B_alpha        : 0.000449[0.000596 0.753 0.451][0.000589 0.762 0.446]
B_alpha_S      : -0.000387[0.0031 -0.125 0.9][0.000607 -0.638 0.523]
B_beta         : 0.00261[6.85e-05 38.1 0][6.8e-05 38.4 0]
B_beta_S       : -0.000747[8.06e-05 -9.26 0][8.16e-05 -9.15 0]
B_g0           : 2.08e-05[1.3e-05 1.6 0.11][1.29e-05 1.61 0.108]
B_g0_S         : -7.36e-06[5.52e-05 -0.133 0.894][1.11e-05 -0.666 0.505]
B_gamma        : 0.00245[6.65e-05 36.9 0][6.54e-05 37.5 0]
B_gamma_S      : -0.000613[9.81e-05 -6.25 4.21e-10][0.000106 -5.76 8.48e-09]
('B_alpha_S', 'B_alpha'):	-3.39e-08	-0.0184	-0.264	0.792	-8.5

In [2]:
df = df[~(df['TRIP_ORIGIN_LOCATION'].isin([151019787])&df['Label'].isin([2]))]
#df = df[~df['GRANT_CARD_CODE'].isin([13513787, 83551908])]
df = df[~(df['TRIP_ORIGIN_LOCATION'].isin([151019789])&df['Label'].isin([1,8]))]
#2df = df[(df['TRIP_ORIGIN_LOCATION'].isin([151019787])&df['Label'].isin([9]))]
#df = df[~(df['TRIP_ORIGIN_LOCATION'].isin([151019789])&df['Label'].isin([1,8]))]
#df = df[df['TRIP_ORIGIN_LOCATION'].isin([151019785,151019784,151019787])]
df

,Label_old,ARRIVETIME_PREFER_old,Label,CHOICE,ARRIVETIME_PREFER,GRANT_CARD_CODE,TRIP_ORIGIN_LOCATION,进站时间,到达时间,出行时长,...,PCC_37,PCC_38,PCC_39,PCC_40,PCC_41,PCC_42,PCC_43,PCC_44,PCC_45,PCC_46
0,0,30300,9,19,30300,1229463,151019784,28680,30249,1569,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,0,30300,9,16,30300,1229463,151019784,28200,29538,1338,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,0,30300,9,22,30300,1229463,151019784,29700,30960,1260,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,0,30300,9,16,30300,1229463,151019784,28200,29534,1334,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0,30300,9,16,30300,1229463,151019784,28140,29536,1396,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7421,1,30900,3,24,30900,89911490,151019787,29340,31297,1957,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7420,1,30900,3,24,30900,89911490,151019787,29460,31305,1845,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7432,1,30900,3,21,30900,89911490,151019787,29040,30760,1720,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7425,1,30900,3,23,30900,89911490,151019787,29460,31282,1822,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
biogeme.modelName='panellogit1228'
results=biogeme.estimate()
pandasResults=results.getEstimatedParameters()
print(pandasResults)

            Value   Std err     t-test   p-value  Rob. Std err  Rob. t-test  \
B_alpha  0.000056  0.000561   0.099418  0.920806      0.000547     0.101987   
B_beta   0.002238  0.000037  59.951853  0.000000      0.000040    56.086792   
B_g0     0.000005  0.000012   0.396635  0.691636      0.000012     0.392008   
B_gamma  0.002246  0.000040  56.276611  0.000000      0.000042    53.631990   

         Rob. p-value  
B_alpha      0.918767  
B_beta       0.000000  
B_g0         0.695053  
B_gamma      0.000000  


In [10]:
biogeme.modelName='panellogit1228'
results=biogeme.estimate()
pandasResults=results.getEstimatedParameters()
print(pandasResults)

            Value   Std err     t-test   p-value  Rob. Std err  Rob. t-test  \
B_alpha  0.001984  0.001214   1.634564  0.102141      0.001224     1.620203   
B_beta   0.002517  0.000079  31.676843  0.000000      0.000081    30.917314   
B_g0    -0.000243  0.000022 -10.891775  0.000000      0.000022   -11.069454   
B_gamma  0.002018  0.000074  27.276853  0.000000      0.000079    25.622890   

         Rob. p-value  
B_alpha      0.105189  
B_beta       0.000000  
B_g0         0.000000  
B_gamma      0.000000  


In [10]:
biogeme.modelName='panellogit1228'
results=biogeme.estimate()
pandasResults=results.getEstimatedParameters()
print(pandasResults)

            Value   Std err     t-test   p-value  Rob. Std err  Rob. t-test  \
B_alpha  0.000332  0.000542   0.613214  0.539735      0.000529     0.627787   
B_beta   0.002300  0.000036  63.631911  0.000000      0.000039    59.504282   
B_g0    -0.000035  0.000012  -3.023626  0.002498      0.000012    -3.001880   
B_gamma  0.002200  0.000037  58.972429  0.000000      0.000039    56.996277   

         Rob. p-value  
B_alpha      0.530144  
B_beta       0.000000  
B_g0         0.002683  
B_gamma      0.000000  


In [10]:
biogeme.modelName='panellogit1228'
results=biogeme.estimate()
pandasResults=results.getEstimatedParameters()
print(pandasResults)

            Value   Std err     t-test   p-value  Rob. Std err  Rob. t-test  \
B_alpha  0.000350  0.000541   0.646439  0.517995      0.000529     0.660986   
B_beta   0.002299  0.000036  63.784970  0.000000      0.000038    59.728877   
B_g0    -0.000037  0.000012  -3.188697  0.001429      0.000012    -3.165348   
B_gamma  0.002187  0.000037  59.004929  0.000000      0.000038    56.871476   

         Rob. p-value  
B_alpha      0.508622  
B_beta       0.000000  
B_g0         0.001549  
B_gamma      0.000000  


In [10]:
print(results)


Results for model panellogit1129
Output file (HTML):			panellogit1129.html
Nbr of parameters:		4
Sample size:			7434
Excluded data:			0
Init log likelihood:		-22065.17
Final log likelihood:		-18510.52
Likelihood ratio test (init):		7109.297
Rho square (init):			0.161
Rho bar square (init):			0.161
Akaike Information Criterion:	37029.05
Bayesian Information Criterion:	37056.7
Final gradient norm:		68.59334
B_alpha        : 0.000371[0.000533 0.696 0.486][0.000532 0.697 0.486]
B_beta         : 0.00149[2.74e-05 54.5 0][2.63e-05 56.9 0]
B_g0           : 4.34e-05[1.1e-05 3.94 8.13e-05][1.13e-05 3.82 0.000132]
B_gamma        : 0.00208[3.63e-05 57.4 0][3.57e-05 58.3 0]
('B_beta', 'B_alpha'):	1.1e-09	0.0756	2.11	0.0346	1.22e-09	0.0875	2.12	0.0342
('B_g0', 'B_alpha'):	-1.75e-09	-0.298	-0.611	0.541	-1.85e-09	-0.306	-0.611	0.541
('B_g0', 'B_beta'):	-1.65e-11	-0.0546	-48.2	0	-4.45e-11	-0.149	-48.2	0
('B_gamma', 'B_alpha'):	-9.61e-10	-0.0496	3.2	0.0014	-4.66e-10	-0.0245	3.21	0.00135
('B_gamma', 'B_

In [12]:
biogeme.modelName='panellogit1124'
results=biogeme.estimate()
pandasResults=results.getEstimatedParameters()
print(pandasResults)

            Value   Std err     t-test       p-value  Rob. Std err  \
B_alpha -0.000216  0.000549  -0.392741  6.945110e-01      0.000548   
B_beta   0.001535  0.000029  52.694518  0.000000e+00      0.000028   
B_g0     0.000073  0.000012   6.334329  2.383762e-10      0.000012   
B_gamma  0.002120  0.000038  56.308530  0.000000e+00      0.000037   

         Rob. t-test  Rob. p-value  
B_alpha    -0.393570  6.938983e-01  
B_beta     54.025073  0.000000e+00  
B_g0        6.082852  1.180634e-09  
B_gamma    57.202039  0.000000e+00  


In [12]:
biogeme.modelName='panellogit1123'
results=biogeme.estimate()
pandasResults=results.getEstimatedParameters()
print(pandasResults)

            Value   Std err     t-test       p-value  Rob. Std err  \
B_alpha -0.000204  0.000545  -0.373728  7.086066e-01      0.000543   
B_beta   0.001520  0.000029  52.922349  0.000000e+00      0.000028   
B_g0     0.000073  0.000011   6.331669  2.425233e-10      0.000012   
B_gamma  0.002100  0.000037  56.483627  0.000000e+00      0.000037   

         Rob. t-test  Rob. p-value  
B_alpha    -0.375203  7.075096e-01  
B_beta     54.448369  0.000000e+00  
B_g0        6.114832  9.665875e-10  
B_gamma    56.858495  0.000000e+00  


In [10]:
biogeme.modelName='panellogit1122'
results=biogeme.estimate()
pandasResults=results.getEstimatedParameters()
print(pandasResults)

            Value   Std err     t-test   p-value  Rob. Std err  Rob. t-test  \
B_alpha  0.000376  0.000536   0.700783  0.483439      0.000535     0.702088   
B_beta   0.001485  0.000028  53.721677  0.000000      0.000027    55.894062   
B_g0     0.000039  0.000011   3.559251  0.000372      0.000011     3.451624   
B_gamma  0.002061  0.000036  56.471565  0.000000      0.000036    57.523079   

         Rob. p-value  
B_alpha      0.482624  
B_beta       0.000000  
B_g0         0.000557  
B_gamma      0.000000  


In [11]:
biogeme.modelName='panellogit85'
results=biogeme.estimate()
pandasResults=results.getEstimatedParameters()
print(pandasResults)

            Value   Std err     t-test   p-value  Rob. Std err  Rob. t-test  \
B_alpha -0.003858  0.001166  -3.309252  0.000935      0.001113    -3.464779   
B_beta   0.001539  0.000059  25.939821  0.000000      0.000055    28.003474   
B_g0     0.000256  0.000026   9.941952  0.000000      0.000023    11.092873   
B_gamma  0.002002  0.000074  27.224701  0.000000      0.000069    29.107551   

         Rob. p-value  
B_alpha      0.000531  
B_beta       0.000000  
B_g0         0.000000  
B_gamma      0.000000  


In [11]:
biogeme.modelName='panellogit87'
results=biogeme.estimate()
pandasResults=results.getEstimatedParameters()
print(pandasResults)

            Value   Std err     t-test   p-value  Rob. Std err  Rob. t-test  \
B_alpha  0.002665  0.000997   2.673961  0.007496      0.000965     2.761139   
B_beta   0.001432  0.000047  30.477187  0.000000      0.000044    32.607239   
B_g0    -0.000067  0.000019  -3.539219  0.000401      0.000019    -3.560658   
B_gamma  0.002227  0.000076  29.326478  0.000000      0.000071    31.200299   

         Rob. p-value  
B_alpha       0.00576  
B_beta        0.00000  
B_g0          0.00037  
B_gamma       0.00000  


In [11]:
biogeme.modelName='panellogit89'
results=biogeme.estimate()
pandasResults=results.getEstimatedParameters()
print(pandasResults)

            Value   Std err     t-test   p-value  Rob. Std err  Rob. t-test  \
B_alpha  0.001771  0.001180   1.501539  0.133216      0.001225     1.445583   
B_beta   0.001677  0.000058  28.841363  0.000000      0.000054    31.332267   
B_g0    -0.000264  0.000022 -12.147271  0.000000      0.000022   -12.158265   
B_gamma  0.002142  0.000083  25.715415  0.000000      0.000087    24.556828   

         Rob. p-value  
B_alpha      0.148294  
B_beta       0.000000  
B_g0         0.000000  
B_gamma      0.000000  


In [9]:
biogeme.modelName='panellogit1'
results=biogeme.estimate()
pandasResults=results.getEstimatedParameters()
print(pandasResults)

            Value   Std err     t-test   p-value  Rob. Std err  Rob. t-test  \
B_alpha -0.000037  0.000531  -0.069264  0.944779      0.000532    -0.069079   
B_beta   0.001519  0.000027  55.644409  0.000000      0.000025    59.660099   
B_g0     0.000044  0.000011   3.965164  0.000073      0.000011     3.840043   
B_gamma  0.002152  0.000037  58.488854  0.000000      0.000035    60.906474   

         Rob. p-value  
B_alpha      0.944927  
B_beta       0.000000  
B_g0         0.000123  
B_gamma      0.000000  


In [10]:
print(results)


Results for model panellogit1
Output file (HTML):			panellogit1.html
Nbr of parameters:		4
Sample size:			7581
Excluded data:			0
Init log likelihood:		-22497.4
Final log likelihood:		-18742.48
Likelihood ratio test (init):		7509.837
Rho square (init):			0.167
Rho bar square (init):			0.167
Akaike Information Criterion:	37492.97
Bayesian Information Criterion:	37520.7
Final gradient norm:		0.003366998
B_alpha        : -3.67e-05[0.000531 -0.0693 0.945][0.000532 -0.0691 0.945]
B_beta         : 0.00152[2.73e-05 55.6 0][2.55e-05 59.7 0]
B_g0           : 4.36e-05[1.1e-05 3.97 7.33e-05][1.13e-05 3.84 0.000123]
B_gamma        : 0.00215[3.68e-05 58.5 0][3.53e-05 60.9 0]
('B_beta', 'B_alpha'):	1.07e-09	0.074	2.94	0.00328	1.29e-09	0.0951	2.93	0.00334
('B_g0', 'B_alpha'):	-1.72e-09	-0.295	0.15	0.88	-1.86e-09	-0.309	0.15	0.881
('B_g0', 'B_beta'):	-1.72e-11	-0.0575	-49.2	0	-4.85e-11	-0.168	-49.9	0
('B_gamma', 'B_alpha'):	-9.68e-10	-0.0496	4.1	4.1e-05	-3.72e-10	-0.0198	4.1	4.13e-05
('B_gamma', 'B_b

In [10]:
for i in range(1,47):
    print('V%s = B_alpha * TRIPTIME_TYPE_%s + B_beta * TE_%s + B_gamma * TL_%s + B_g0 * crowding_cost_%s + B_fixed_%s '%(i,i,i,i,i,i))

V1 = B_alpha * TRIPTIME_TYPE_1 + B_beta * TE_1 + B_gamma * TL_1 + B_g0 * crowding_cost_1 + B_fixed_1 
V2 = B_alpha * TRIPTIME_TYPE_2 + B_beta * TE_2 + B_gamma * TL_2 + B_g0 * crowding_cost_2 + B_fixed_2 
V3 = B_alpha * TRIPTIME_TYPE_3 + B_beta * TE_3 + B_gamma * TL_3 + B_g0 * crowding_cost_3 + B_fixed_3 
V4 = B_alpha * TRIPTIME_TYPE_4 + B_beta * TE_4 + B_gamma * TL_4 + B_g0 * crowding_cost_4 + B_fixed_4 
V5 = B_alpha * TRIPTIME_TYPE_5 + B_beta * TE_5 + B_gamma * TL_5 + B_g0 * crowding_cost_5 + B_fixed_5 
V6 = B_alpha * TRIPTIME_TYPE_6 + B_beta * TE_6 + B_gamma * TL_6 + B_g0 * crowding_cost_6 + B_fixed_6 
V7 = B_alpha * TRIPTIME_TYPE_7 + B_beta * TE_7 + B_gamma * TL_7 + B_g0 * crowding_cost_7 + B_fixed_7 
V8 = B_alpha * TRIPTIME_TYPE_8 + B_beta * TE_8 + B_gamma * TL_8 + B_g0 * crowding_cost_8 + B_fixed_8 
V9 = B_alpha * TRIPTIME_TYPE_9 + B_beta * TE_9 + B_gamma * TL_9 + B_g0 * crowding_cost_9 + B_fixed_9 
V10 = B_alpha * TRIPTIME_TYPE_10 + B_beta * TE_10 + B_gamma * TL_10 + B_g0 * crowd

In [12]:
for i in range(1,22):
    print('V%s = B_alpha * TRIPTIME_%s + B_beta * TE_%s + B_gamma * TL_%s + B_g0 * 1680 * flow_normal_dic[%s] '%(i,i,i,i,i))

V1 = B_alpha * TRIPTIME_1 + B_beta * TE_1 + B_gamma * TL_1 + B_g0 * 1680 * flow_normal_dic[1] 
V2 = B_alpha * TRIPTIME_2 + B_beta * TE_2 + B_gamma * TL_2 + B_g0 * 1680 * flow_normal_dic[2] 
V3 = B_alpha * TRIPTIME_3 + B_beta * TE_3 + B_gamma * TL_3 + B_g0 * 1680 * flow_normal_dic[3] 
V4 = B_alpha * TRIPTIME_4 + B_beta * TE_4 + B_gamma * TL_4 + B_g0 * 1680 * flow_normal_dic[4] 
V5 = B_alpha * TRIPTIME_5 + B_beta * TE_5 + B_gamma * TL_5 + B_g0 * 1680 * flow_normal_dic[5] 
V6 = B_alpha * TRIPTIME_6 + B_beta * TE_6 + B_gamma * TL_6 + B_g0 * 1680 * flow_normal_dic[6] 
V7 = B_alpha * TRIPTIME_7 + B_beta * TE_7 + B_gamma * TL_7 + B_g0 * 1680 * flow_normal_dic[7] 
V8 = B_alpha * TRIPTIME_8 + B_beta * TE_8 + B_gamma * TL_8 + B_g0 * 1680 * flow_normal_dic[8] 
V9 = B_alpha * TRIPTIME_9 + B_beta * TE_9 + B_gamma * TL_9 + B_g0 * 1680 * flow_normal_dic[9] 
V10 = B_alpha * TRIPTIME_10 + B_beta * TE_10 + B_gamma * TL_10 + B_g0 * 1680 * flow_normal_dic[10] 
V11 = B_alpha * TRIPTIME_11 + B_beta * TE_11 

In [13]:
for i in range(22,47):
    print('V%s = B_alpha * TRIPTIME_%s + B_beta * TE_%s + B_gamma * TL_%s + B_g0 * 1680 * flow_normal_dic[%s] '%(i,i,i,i,i))

V22 = B_alpha * TRIPTIME_22 + B_beta * TE_22 + B_gamma * TL_22 + B_g0 * 1680 * flow_normal_dic[22] 
V23 = B_alpha * TRIPTIME_23 + B_beta * TE_23 + B_gamma * TL_23 + B_g0 * 1680 * flow_normal_dic[23] 
V24 = B_alpha * TRIPTIME_24 + B_beta * TE_24 + B_gamma * TL_24 + B_g0 * 1680 * flow_normal_dic[24] 
V25 = B_alpha * TRIPTIME_25 + B_beta * TE_25 + B_gamma * TL_25 + B_g0 * 1680 * flow_normal_dic[25] 
V26 = B_alpha * TRIPTIME_26 + B_beta * TE_26 + B_gamma * TL_26 + B_g0 * 1680 * flow_normal_dic[26] 
V27 = B_alpha * TRIPTIME_27 + B_beta * TE_27 + B_gamma * TL_27 + B_g0 * 1680 * flow_normal_dic[27] 
V28 = B_alpha * TRIPTIME_28 + B_beta * TE_28 + B_gamma * TL_28 + B_g0 * 1680 * flow_normal_dic[28] 
V29 = B_alpha * TRIPTIME_29 + B_beta * TE_29 + B_gamma * TL_29 + B_g0 * 1680 * flow_normal_dic[29] 
V30 = B_alpha * TRIPTIME_30 + B_beta * TE_30 + B_gamma * TL_30 + B_g0 * 1680 * flow_normal_dic[30] 
V31 = B_alpha * TRIPTIME_31 + B_beta * TE_31 + B_gamma * TL_31 + B_g0 * 1680 * flow_normal_dic[31] 


In [ ]:
"""File 12panel.py

:author: Michel Bierlaire, EPFL
:date: Sun Sep  8 18:55:38 2019

 Example of a mixture of logit models, using Monte-Carlo integration.
 The datafile is organized as panel data.
 Three alternatives: Train, Car and Swissmetro
 SP data
"""
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
from biogeme.expressions import Beta, DefineVariable, bioDraws, PanelLikelihoodTrajectory, MonteCarlo, log

# Read the data
df = pd.read_csv("swissmetro.dat",sep='\t')
database = db.Database("swissmetro",df)

# They are organized as panel data. The variable ID identifies each individual.
database.panel("ID")

# The Pandas data structure is available as database.data. Use all the
# Pandas functions to invesigate the database
#print(database.data.describe())

# The following statement allows you to use the names of the variable
# as Python variable.
globals().update(database.variables)

# Removing some observations can be done directly using pandas.
#remove = (((database.data.PURPOSE != 1) & (database.data.PURPOSE != 3)) | (database.data.CHOICE == 0))
#database.data.drop(database.data[remove].index,inplace=True)

# Here we use the "biogeme" way for backward compatibility
exclude = (( PURPOSE != 1 ) * (  PURPOSE   !=  3  ) +  ( CHOICE == 0 )) > 0
database.remove(exclude)

# Parameters to be estimated
ASC_CAR = Beta('ASC_CAR',0,None,None,0)
ASC_TRAIN = Beta('ASC_TRAIN',0,None,None,0)
ASC_SM = Beta('ASC_SM',0,None,None,1)
B_TIME = Beta('B_TIME',0,None,None,0)
B_COST = Beta('B_COST',0,None,None,0)
B_TIME_S = Beta('B_TIME_S',0,None,None,0)

# Define a random parameter, normally distributed across individuals,
# designed to be used for Monte-Carlo simulation
B_TIME_RND = B_TIME + B_TIME_S * bioDraws('B_TIME_RND','NORMAL')

# Definition of new variables
SM_COST =  SM_CO   * (  GA   ==  0  ) 
TRAIN_COST =  TRAIN_CO   * (  GA   ==  0  )

# Definition of new variables: adding columns to the database 
TRAIN_TT_SCALED = DefineVariable('TRAIN_TT_SCALED',\
                                 TRAIN_TT / 100.0,database)
TRAIN_COST_SCALED = DefineVariable('TRAIN_COST_SCALED',\
                                   TRAIN_COST / 100,database)
SM_TT_SCALED = DefineVariable('SM_TT_SCALED', SM_TT / 100.0,database)
SM_COST_SCALED = DefineVariable('SM_COST_SCALED', SM_COST / 100,database)
CAR_TT_SCALED = DefineVariable('CAR_TT_SCALED', CAR_TT / 100,database)
CAR_CO_SCALED = DefineVariable('CAR_CO_SCALED', CAR_CO / 100,database)

# Definition of the utility functions
V1 = ASC_TRAIN + \
     B_TIME_RND * TRAIN_TT_SCALED + \
     B_COST * TRAIN_COST_SCALED
V2 = ASC_SM + \
     B_TIME_RND * SM_TT_SCALED + \
     B_COST * SM_COST_SCALED
V3 = ASC_CAR + \
     B_TIME_RND * CAR_TT_SCALED + \
     B_COST * CAR_CO_SCALED

# Associate utility functions with the numbering of alternatives
V = {1: V1,
     2: V2,
     3: V3}

# Associate the availability conditions with the alternatives
CAR_AV_SP =  DefineVariable('CAR_AV_SP',CAR_AV  * (  SP   !=  0  ),database)
TRAIN_AV_SP =  DefineVariable('TRAIN_AV_SP',TRAIN_AV  * (  SP   !=  0  ),database)
av = {1: TRAIN_AV_SP,
      2: SM_AV,
      3: CAR_AV_SP}

# Conditional to B_TIME_RND, the likelihood of one observation is
# given by the logit model (called the kernel)
obsprob = models.logit(V,av,CHOICE)

# Conditional to B_TIME_RND, the likelihood of all observations for
# one individual (the trajectory) is the product of the likelihood of
# each observation.
condprobIndiv = PanelLikelihoodTrajectory(obsprob)

# We integrate over B_TIME_RND using Monte-Carlo
logprob = log(MonteCarlo(condprobIndiv))

# Define level of verbosity
import biogeme.messaging as msg
logger = msg.bioMessage()
#logger.setSilent()
#logger.setWarning()
#logger.setGeneral()
logger.setDetailed()
#logger.setDebug()

# Create the Biogeme object
biogeme  = bio.BIOGEME(database,logprob,numberOfDraws=5000)
biogeme.modelName = "12panel"

# Estimate the parameters. 
results = biogeme.estimate()
pandasResults = results.getEstimatedParameters()
print(pandasResults)